In [6]:
import os

def loadFAQs(directory_path):
   faqs = {}

   for filename in os.listdir(directory_path):
      if filename.endswith(".txt"):  # assuming FAQs are in .txt files
         file_path = os.path.join(directory_path, filename)

         with open(file_path) as f:
            raw_faq = f.read()

         filename_without_ext = os.path.splitext(filename)[0]  # remove .txt extension
         faqs[filename_without_ext] = [text.strip() for text in raw_faq.split('=====')]

   return faqs

In [7]:
faqs = loadFAQs('.')
faqs

{'faq': ['Who are you and what can you do?\n\nI am DORA, the Digital ORacle Assistant, a digital assistant working for Oracle EMEA. I can answer questions about Oracle Cloud (OCI) and especially about the Free Trial and Always Free programs.',
  'What is Oracle Cloud Free Tier?\n\nOracle Cloud Free Tier allows you to sign up for an Oracle Cloud account which provides a number of Always Free services and a Free Trial with US$300 of free credit to use on all eligible Oracle Cloud Infrastructure services for up to 30 days. The Always Free services are available for an unlimited period of time. The Free Trial services may be used until your US$300 of free credits are consumed or the 30 days has expired, whichever comes first.',
  'Who should use Oracle Cloud Free Tier?\n\nOracle Cloud Free Tier services are for everyone. Whether you’re a developer building and testing applications, a startup founder creating new systems with the intention of scaling later, an enterprise looking to test thi

In [8]:
docs = [{'text': filename + ' | ' + section, 'path': filename} for filename, sections in faqs.items() for section in sections]

# Sample the resulting data
docs[:2]

[{'text': 'faq | Who are you and what can you do?\n\nI am DORA, the Digital ORacle Assistant, a digital assistant working for Oracle EMEA. I can answer questions about Oracle Cloud (OCI) and especially about the Free Trial and Always Free programs.',
  'path': 'faq'},
 {'text': 'faq | What is Oracle Cloud Free Tier?\n\nOracle Cloud Free Tier allows you to sign up for an Oracle Cloud account which provides a number of Always Free services and a Free Trial with US$300 of free credit to use on all eligible Oracle Cloud Infrastructure services for up to 30 days. The Always Free services are available for an unlimited period of time. The Free Trial services may be used until your US$300 of free credits are consumed or the 30 days has expired, whichever comes first.',
  'path': 'faq'}]

In [9]:
import oracledb

from tqdm.autonotebook import tqdm, trange



/tmp/ipykernel_11094/3746924790.py:3: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [10]:
connection = oracledb.connect(
    user="admin",
    password="Oracle123...",
    dsn="(description= (retry_count=20)(retry_delay=3)(address=(protocol=tcps)(port=1521)(host=adb.us-chicago-1.oraclecloud.com))(connect_data=(service_name=g1cde62092a80c9_pwyxg3f50l3b80w9_high.adb.oraclecloud.com))(security=(ssl_server_dn_match=yes)))")  # the connection string copied from the cloud console

print("Successfully connected to Oracle Database")

Successfully connected to Oracle Database


In [11]:
table_name = 'faqs'

with connection.cursor() as cursor:
    # Create the table
    create_table_sql = f"""
        CREATE TABLE IF NOT EXISTS {table_name} (
            id NUMBER PRIMARY KEY,
            payload CLOB CHECK (payload IS JSON),
            vector VECTOR
        )"""
    try:
        cursor.execute(create_table_sql)
    except oracledb.DatabaseError as e:
        raise

    connection.autocommit = True

In [12]:
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer('all-MiniLM-L12-v2')

/home/ubuntu/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
import array

# Define a list to store the data
data = [
   {"id": idx, "vector_source": row['text'], "payload": row} 
   for idx, row in enumerate(docs)
]

# Collect all texts for batch encoding
texts = [f"{row['vector_source']}" for row in data]

# Encode all texts in a batch
embeddings = encoder.encode(texts, batch_size=32, show_progress_bar=True)

# Assign the embeddings back to your data structure
for row, embedding in zip(data, embeddings):
   row['vector'] = array.array("f", embedding)

Batches: 100%|██████████| 7/7 [00:16<00:00,  2.41s/it]


In [14]:
import json

with connection.cursor() as cursor:
    # Truncate the table
    cursor.execute(f"truncate table {table_name}")

    prepared_data = [(row['id'], json.dumps(row['payload']), row['vector']) for row in data]

    # Insert the data
    cursor.executemany(
        f"""INSERT INTO {table_name} (id, payload, vector)
        VALUES (:1, :2, :3)""",
        prepared_data
    )

    connection.commit()

In [15]:
with connection.cursor() as cursor:
    # Define the query to select all rows from a table
    query = f"SELECT * FROM {table_name}"

    # Execute the query
    cursor.execute(query)

    # Fetch all rows
    rows = cursor.fetchall()

    # Print the rows
    for row in rows[:5]:
        print(row)

(2, {'text': 'faq | Who should use Oracle Cloud Free Tier?\n\nOracle Cloud Free Tier services are for everyone. Whether you’re a developer building and testing applications, a startup founder creating new systems with the intention of scaling later, an enterprise looking to test things before moving to cloud, a student wanting to learn, or an academic developing curriculum in the cloud, Oracle Cloud Free Tier enables you to learn, explore, build and test for free.', 'path': 'faq'}, array('f', [0.047548577189445496, -0.022579016163945198, 0.01457521878182888, -0.021478641778230667, 0.01285360474139452, -0.01365294586867094, -0.07855579257011414, 0.05885817110538483, -0.003879772499203682, 0.0028984397649765015, 0.02300197258591652, 0.021663499996066093, -0.014754826202988625, -0.052749719470739365, 0.029338199645280838, -0.059429798275232315, 0.020581789314746857, 0.0016599860973656178, 0.00407831184566021, -0.05929175391793251, -0.04914326220750809, -0.030911842361092567, -0.0242928992

In [16]:
topK = 3

sql = f"""select payload, vector_distance(vector, :vector, COSINE) as score
from {table_name}
order by score
fetch approx first {topK} rows only"""

In [17]:
question = 'What is Always Free?'

In [18]:
with connection.cursor() as cursor:
  embedding = list(encoder.encode(question))
  vector = array.array("f", embedding)

In [19]:
with connection.cursor() as cursor:
  embedding = list(encoder.encode(question))
  vector = array.array("f", embedding)

  results  = []

  for (info, score, ) in cursor.execute(sql, vector=vector):
      text_content = info.read()
      results.append((score, json.loads(text_content)))

In [20]:
import pprint
pprint.pp(results)

[(0.34205919353536807,
  {'text': 'faq | What are Always Free services?\n'
           '\n'
           'Always Free services are part of Oracle Cloud Free Tier. Always '
           'Free services are available for an unlimited time. Some '
           'limitations apply. As new Always Free services become available, '
           'you will automatically be able to use those as well.\n'
           '\n'
           'The following services are available as Always Free:\n'
           '\n'
           'AMD-based Compute\n'
           'Arm-based Ampere A1 Compute\n'
           'Block Volume\n'
           'Object Storage\n'
           'Archive Storage\n'
           'Flexible Load Balancer\n'
           'Flexible Network Load Balancer\n'
           'VPN Connect\n'
           'Autonomous Data Warehouse\n'
           'Autonomous Transaction Processing\n'
           'Autonomous JSON Database\n'
           'NoSQL Database (Phoenix Region only)\n'
           'APEX Application Development\n'
           '

In [27]:
from transformers import LlamaTokenizerFast
import sys

tokenizer = LlamaTokenizerFast.from_pretrained("hf-internal-testing/llama-tokenizer")


tokenizer.model_max_length = sys.maxsize

def truncate_string(string, max_tokens):
    # Tokenize the text and count the tokens
    tokens = tokenizer.encode(string, add_special_tokens=True) 
    # Truncate the tokens to a maximum length
    truncated_tokens = tokens[:max_tokens]
    # transform the tokens back to text
    truncated_text = tokenizer.decode(truncated_tokens)
    return truncated_text

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [28]:
# transform docs into a string array using the "paylod" key
docs_as_one_string = "\n=========\n".join([doc["text"] for doc in docs])
docs_truncated = truncate_string(docs_as_one_string, 1000)

In [19]:
prompt = f"""\
    <s>[INST] <<SYS>>
    You are a helpful assistant named Oracle chatbot. 
    USE ONLY the sources below and ABSOLUTELY IGNORE any previous knowledge.
    Use Markdown if appropriate.
    Assume the customer is highly technical.
    <</SYS>> [/INST]

    [INST]
    Respond to PRECISELY to this question: "{question}.",  USING ONLY the following information and IGNORING ANY PREVIOUS KNOWLEDGE.
    Include code snippets and commands where necessary.
    NEVER mention the sources, always respond as if you have that knowledge yourself. Do NOT provide warnings or disclaimers.
    =====
    Sources: {docs_truncated}
    =====
    Answer (Three paragraphs, maximum 50 words each, 90% spartan):
    [/INST]
    """

In [5]:
import oci
from oci.auth import signers

# Compartment ID
compartment_id = "ocid1.compartment.oc1..aaaaaaaadbc47j6m3jv3ov2g4ladbrpg6spc2sycw7qiubdqc4eysi664byq"

# Crear un proveedor de autenticación de instancias principales
signer = oci.auth.signers.InstancePrincipalsSecurityTokenSigner()

# Service endpoint
endpoint = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"

# GenAI client usando instance principals
generative_ai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(
    config={}, 
    signer=signer,
    service_endpoint=endpoint,
    retry_strategy=oci.retry.NoneRetryStrategy(),
    timeout=(10,240)
)

In [6]:
prompt = ' HI'

In [ ]:
chat_detail = oci.generative_ai_inference.models.ChatDetails()

chat_request = oci.generative_ai_inference.models.CohereChatRequest()
chat_request.message = prompt
chat_request.max_tokens = 1000
chat_request.temperature = 0.0
chat_request.frequency_penalty = 0
chat_request.top_p = 0.75
chat_request.top_k = 0

chat_detail.serving_mode = oci.generative_ai_inference.models.OnDemandServingMode(model_id="ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceyaycmwwnvu2gaqrffquofgmshlqzcdwpk727n4cykg34oa")
chat_detail.chat_request = chat_request
chat_detail.compartment_id = compartment_id
chat_response = generative_ai_inference_client.chat(chat_detail)

pprint.pp(
    chat_response.data.chat_response.chat_history[1].message
)

In [ ]:
import oci
from oci.auth import signers

# Compartment ID
compartment_id = "ocid1.compartment.oc1..aaaaaaaadbc47j6m3jv3ov2g4ladbrpg6spc2sycw7qiubdqc4eysi664byq"

# Crear un proveedor de autenticación de instancias principales
signer = oci.auth.signers.InstancePrincipalsSecurityTokenSigner()

In [21]:
import oci

compartment_id = "ocid1.compartment.oc1..aaaaaaaadbc47j6m3jv3ov2g4ladbrpg6spc2sycw7qiubdqc4eysi664byq"
CONFIG_PROFILE = "DEFAULT"
config = oci.config.from_file('config', CONFIG_PROFILE)

# Service endpoint
endpoint = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"

# GenAI client
generative_ai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(config=config, service_endpoint=endpoint, retry_strategy=oci.retry.NoneRetryStrategy(), timeout=(10,240))

In [30]:
chat_detail = oci.generative_ai_inference.models.ChatDetails()

chat_request = oci.generative_ai_inference.models.CohereChatRequest()
chat_request.message = prompt
chat_request.max_tokens = 1000
chat_request.temperature = 0.0
chat_request.frequency_penalty = 0
chat_request.top_p = 0.75
chat_request.top_k = 0

chat_detail.serving_mode = oci.generative_ai_inference.models.OnDemandServingMode(model_id="cohere.command-r-plus")
chat_detail.chat_request = chat_request
chat_detail.compartment_id = compartment_id
chat_response = generative_ai_inference_client.chat(chat_detail)

pprint.pp(
    chat_response.data.chat_response.chat_history[1].message
)

('Oracle Cloud Infrastructure offers an Always Free tier which provides access '
 'to a range of services free of charge and without time limits. This includes '
 'AMD and Arm-based compute options, Block Volume, Object Storage, and Archive '
 'Storage. The Always Free tier is intended to cater to a diverse range of '
 'users, from developers and startups to enterprises and students, offering an '
 'opportunity to explore, build, and test on Oracle Cloud at no cost.\n'
 '\n'
 'The specific Always Free services on offer are:\n'
 '\n'
 '- AMD-based Compute\n'
 '- Arm-based Ampere A1 Compute\n'
 '- Block Volume\n'
 '- Object Storage\n'
 '- Archive Storage\n'
 '\n'
 'These services are available without any time restrictions, providing an '
 'excellent opportunity to familiarize oneself with Oracle Cloud '
 'Infrastructure.')
